<a href="https://colab.research.google.com/github/LukaLujan/word2vec/blob/main/gthb2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This notebook is very similar like one before, so there will be less comments. 

import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
import gensim.downloader as api
from google.colab import drive

import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

from keras.preprocessing.text import Tokenizer


import re
STOPWORDS = set(stopwords.words('english'))

import seaborn as sns

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#This where I am going to mount my drive on google colab
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#I am changing directory , however you may do all of this on different way. 
%cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
#i have this predownloaded on my google drive. This is a link and code to download pretrained model. 

#!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
#this is how you activate your model. notice that it will take huge % of your google colab RAM. We will have to deal with it.

EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
word2vec_model = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
v_apple = word2vec_model["apple"] 

print(v_apple.shape)

(300,)


In [ ]:

word2vec_model["apple"]


In [ ]:

#word2vec_model["breitbart"]   -not in vocab

In [ ]:
#Now process is almost the same as in the notebook before.
df =pd.read_csv("train.csv")
df=df.dropna()

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer=WordNetLemmatizer()

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#Preprocess our "titles" - we dont need to preprocess df["text"] because our model is already pre-trained.
messages_text = []

tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
for par in df["title"].values:
    tmp = []
    sentences = nltk.sent_tokenize(par)
     
    for sent in sentences:
      
        sent = sent.lower()
        
        tokens = tokenizer.tokenize(sent)
         

        filtered_words = [w.strip() for w in tokens if w not in STOPWORDS and len(w) > 1]
        filtered_words2 = [lemmatizer.lemmatize(w) for w in filtered_words]
        tmp.extend(filtered_words2)
    messages_text.append(tmp)

In [ ]:
def ones_vector(d):
  return np.ones(d)
  

In [ ]:
for i in range(len(messages_text)):
  messages_text[i] =" ".join(word for word in messages_text[i])

In [ ]:
lista_vectora =[]
for row in messages_text:
  temp_ls_vec=[]
  for token in row.split():
    if token in word2vec_model.vocab:
      temp_ls_vec.append(word2vec_model[token])
    else:
      temp_ls_vec.append(ones_vector(300))
  lista_vectora.append(temp_ls_vec)

In [ ]:
max_len =[]
for sent in lista_vectora:
  max_len.append(len(sent))
print(max(max_len))
print(len(max_len))
L = max(max_len)

47
18285


In [ ]:
def null_vector(num):
  return np.zeros(num)

In [ ]:
padded_embeddings = []
for row in lista_vectora:
  temp_zer=[]
  if len(row) <L:
    for i in range(L-len(row)):
      temp_zer.append(null_vector(300))
  padded_embeddings.append(temp_zer)  

In [ ]:
padded_vectors= [ k+v for k,v in zip(lista_vectora ,padded_embeddings )]

In [ ]:
len(padded_vectors)

18285

In [ ]:

y_train = np.array(df["label"])
X_train = np.array(padded_vectors)

In [ ]:
X_train.shape

(18285, 47, 300)

In [ ]:

input_shape =(X_train.shape[1], X_train.shape[2])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Spliting data into validation and train. Test data will come latter.
X1_train, X1_val, y1_train, y1_val  =train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Input, Dense, LSTM, SpatialDropout1D, Dropout, TimeDistributed, Input
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()
# model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
    # here we do not need Embedding layer because we did it on our own) 
model.add(Input(shape=input_shape))
    # https://keras.io/api/layers/core_layers/input/

#This is my simple network that will yield some results. Goal here was more on preprocessing, embedding and how to use Word2Vec.
#You can play with tunning 

model.add(LSTM(64))
model.add(Dropout(0.4))
model.add(Dense(32))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                93440     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 95,553
Trainable params: 95,553
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# I am deleting all those variables to release RAM. Otherwise my google-colab 12GB RAM will crash. 
del messages_text, padded_embeddings, padded_vectors, df, lista_vectora, X_train, y_train

In [ ]:

model.fit(X1_train,y1_train, validation_data=(X1_val, y1_val), epochs=5, batch_size=32, callbacks=[EarlyStopping(monitor='val_loss', patience=2,restore_best_weights=True)] )

Epoch 1/5
383/383 [==============================] - 22s 51ms/step - loss: 0.3846 - accuracy: 0.8218 - val_loss: 0.3203 - val_accuracy: 0.8767
Epoch 2/5
383/383 [==============================] - 20s 52ms/step - loss: 0.2715 - accuracy: 0.8887 - val_loss: 0.3089 - val_accuracy: 0.8560
Epoch 3/5
383/383 [==============================] - 19s 50ms/step - loss: 0.2316 - accuracy: 0.9038 - val_loss: 0.2529 - val_accuracy: 0.8936
Epoch 4/5
383/383 [==============================] - 20s 52ms/step - loss: 0.2086 - accuracy: 0.9109 - val_loss: 0.2361 - val_accuracy: 0.8996
Epoch 5/5
383/383 [==============================] - 19s 50ms/step - loss: 0.1912 - accuracy: 0.9215 - val_loss: 0.2508 - val_accuracy: 0.9024


In [ ]:
#Importing our data. Notace that  y -values are in separated csv file("submit.csv")
df_Xtest = pd.read_csv("test.csv")
df_ytest =pd.read_csv("submit.csv")

In [ ]:
#concating two separated test data frames so I can for example get rid of rows with missing values much easier.
df_Xy = pd.concat([df_Xtest, df_ytest], axis=1)

In [ ]:

df_Xy.dropna(inplace=True)
df_Xy.reset_index(inplace=True)
print(len(df_Xy))

4575


In [ ]:
#test data must pass same process as train data. Everything must be preprocessed and tokens converted into 300 dimensional vectors.

messagesX = []

tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
for par in df_Xy["title"].values:
    tmp = []
    sentences = nltk.sent_tokenize(par)
    for sent in sentences:
        sent = sent.lower()
        tokens = tokenizer.tokenize(sent)
        filtered_words = [w.strip() for w in tokens if w not in STOPWORDS and len(w) > 1]
        filtered_words2 = [lemmatizer.lemmatize(w) for w in filtered_words]
        tmp.extend(filtered_words2)
    messagesX.append(tmp)

In [ ]:
messagesX[0]

['specter',
 'trump',
 'loosens',
 'tongue',
 'purse',
 'string',
 'silicon',
 'valley',
 'new',
 'york',
 'time']

In [ ]:
for i in range(len(messagesX)):
 messagesX[i] =" ".join(word for word in messagesX[i]) 

In [ ]:
#if the word from data set is part of our vocab(that is trained on text data ) it will get it's 300 dimension vector. Otherwise it gets vector of ones.
lista_vectora2 =[]
for row in messagesX:
  temp_ls_vec=[]
  for token in row.split():
    if token in word2vec_model.wv.vocab:
      temp_ls_vec.append(word2vec_model.wv[token])
    else:
      temp_ls_vec.append(ones_vector(300))
  lista_vectora2.append(temp_ls_vec)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


In [ ]:
#Everything else that is "empty space" will be padded with zeros.
padded_embeddings2 = []
for row in lista_vectora2:
  temp_zer=[]
  if len(row) <L:
    for i in range(L-len(row)):
      temp_zer.append(null_vector(300))
  padded_embeddings2.append(temp_zer) 

In [ ]:
#as with train data, we do same with test data
padded_vectors2= [ k+v for k,v in zip(lista_vectora2 ,padded_embeddings2 )]

In [ ]:
y_test = np.array(df_Xy["label"])
X_test = np.array(padded_vectors2)

In [ ]:
#predicting results
y_preds = model.predict(X_test)

In [ ]:
#because I used sigmoid function for predictions, everything above 0.5 will be classifed as fake news, everything bellow 0.5 as reliable news source
for i in range(len(y_preds)):
  if y_preds[i] >=0.5:
    y_preds[i]=1
  else:
    y_preds[i] =0

In [ ]:
#lets see how our data is split
y_preds=y_preds.flatten()
pd.Series(y_preds).value_counts()

0.0    2677
1.0    1898
dtype: int64

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
#as we can see, accuracy is even lower in this huge pre-trained model.
print(accuracy_score(y_test, y_preds))

0.6192349726775956


In next notebook we will combine word2vec model that will train and learn on our corpus and with pre-trained word2vec model. 